# Recomendation System

### Scrape data from user's Instagram

In [ ]:
!pip3 install selenium
!apt-get -q update
!apt install -yq chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

import pandas as pd
driver = webdriver.Chrome('chromedriver', options=chrome_options)

!pip3 install instaloader
from instaloader import instaloader, Profile
import time
sys.setrecursionlimit(10000)
from datetime import datetime
from itertools import dropwhile, takewhile



Ask user for their username and password to get access to user's instagram data.

In [ ]:
L = instaloader.Instaloader()
L.login("username","password") #username and log in here separated by comma for instagram account
df=pd.DataFrame()

Collect data of every post on user's account such as caption, likes, comments and URl to post.

In [ ]:
i=0
for post in instaloader.Profile.from_username(L.context, 'user').get_posts():
    df = df.append({'Caption': post.caption, 'Likes':post.likes, 'URL':post.url,'Comments':post.comments},ignore_index=True)
    i=i+1
    if i>400:
        break   
df.to_excel("user.xlsx",index=False)
print("Written to user.xlsx file")


Create and input URLs into an excel sheet.

In [ ]:
#Excel file with URL only
df2=df.drop(columns=['Caption','Likes','Comments'])
df2.to_excel("user_mod.xlsx",index=False)

Call google cloud vision API.

In [ ]:
#!pip install --upgrade google-api-python-client
!pip install google-cloud    
!pip install google-cloud-vision
import xlrd
from google.cloud import vision
import os
import pandas as pd

Send every URL to google vision to retrieve labels describing image and add the labels to the excel sheet.

In [ ]:
Application_Credentials = '.json' #Add API key here from Google Cloud Vision, its a .json file
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = Application_Credentials
client = vision.ImageAnnotatorClient()
image = vision.Image() 

loc= ("user_mod.xlsx")
wb= xlrd.open_workbook(loc)
sheet = wb.sheet_by_index(0)
sheet.cell_value(0, 0)
df3=pd.DataFrame()

#loop through every url, retrieve the image  and send to google vision
for i in range (sheet.nrows):
    image_src_temp = sheet.cell_value(i,0)
    image.source.image_uri = image_src_temp
    response = client.label_detection(image=image)
    labels = response.label_annotations
    l=[]
    for label in labels:
        l.append(label.description)
    s= ' '.join(l)
    print("s")
    print(s)
    df3=df3.append({'URL': image_src_temp, 'Labels':s}, ignore_index = True)
df3.to_excel("Labels.xlsx", index=False)

In [ ]:
df3_copy = df3.copy()
df3_copy = df3_copy.iloc[1:]
df3_copy.reset_index(drop=True, inplace=True)
df3_copy

In [ ]:
#combine labels with origional excel file
df['Labels']=df3_copy['Labels']

In [ ]:
df

In [ ]:
df.to_excel("Combined.xlsx",index=False)

In [10]:
import pandas as pd
import nltk, lda
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
import warnings
import re
import numpy as np
warnings.filterwarnings("ignore")
import logging
logging.getLogger("requests").setLevel(logging.WARNING)

Fix formatting of labels to keep it universal throughout the application. Create 5 groups to divide labels and weight each label on how closely it correlates with every individual group. Create a nrw excel sheet with labels, URl and weightage for every group.

In [ ]:
labels = df["Labels"]
def combine_bigram(labels): 
    tokens = re.findall('[A-Z][^A-Z]*', labels)
    tokens = [t.replace(' ','') for t in tokens]
    return ' '.join(tokens)

def tokenize_text(image_labels):
    word_tokenizer = RegexpTokenizer(r'\w+')
    return word_tokenizer.tokenize(image_labels)

labels = labels.apply(combine_bigram)
num_topic = 5
vec_words = CountVectorizer(tokenizer =tokenize_text, decode_error='ignore')
feature_words = vec_words.fit_transform(labels)

model = lda.LDA(n_topics = num_topic, n_iter = 500,random_state = 1)
model.fit(feature_words)

In [12]:
topic_word = model.topic_word_
doc_topic = model.doc_topic_
doc_topic = pd.DataFrame(doc_topic)
df = df.join(doc_topic)


topics = pd.DataFrame(topic_word)
topics.columns = vec_words.get_feature_names()
topic_transpose = topics.transpose()
topic_transpose.to_excel("topic_word.xlsx")
df.to_excel("doc_topic.xlsx")

In [ ]:
n_top_words = 10
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vec_words.get_feature_names())[np.argsort(topic_dist)][:-n_top_words:-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

List of groups and their description.

group 1: outdoors/adventure

group 2: sightseeing

group 3: music

group 4: arts

group 5: sports

Display the data.

In [ ]:
#assign group names
df = df.rename(columns={0: 'outdoors', 1: 'sightseeing', 2:'music',3:'arts',4:'sports'})

output = df
output.head()